In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input
import pandas as pd



# ------------------------------------------------------
# LOAD DATA
# ------------------------------------------------------
df = pd.read_csv("data.csv")

X = tf.constant(
    df[["math", "science", "english", "attendance"]].values,
    dtype=tf.float32
)
# X = X / 100.0
y = tf.constant(
    df[["label"]].values,   # keep shape (N, 1)
    dtype=tf.float32
)

print("STEP 1: Data shapes")
print("X shape:", X.shape)  # (num_samples, 4)
print("y shape:", y.shape)  # (num_samples, 1)
print("-" * 60)


# ------------------------------------------------------
# STEP 2: Build the model (Sequential = layers in a straight line)
# ------------------------------------------------------
# Model design:
#   Input(shape=(4,))         -> each sample has 4 features
#   Dense(8, relu)            -> learns patterns/feature combinations
#   Dense(1, sigmoid)         -> outputs probability of Pass (0 to 1)
model = Sequential([
    Input(shape=(4,)),                 # Explicit input layer (best practice)
    Dense(8, activation="relu"),       # Hidden layer
    Dense(8, activation="relu"),       # Hidden layer (optional
    Dense(1, activation="sigmoid")     # Output layer for binary classification
])

print("STEP 2: Model summary (shows shape flow)")
model.summary()
print("-" * 60)


# ------------------------------------------------------
# STEP 3: Compile the model
# ------------------------------------------------------
# optimizer: how weights are updated (Adam is a good default)
# loss: binary_crossentropy is used for binary classification (Pass/Fail)
# metrics: accuracy to see performance during training
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

print("STEP 3: Model compiled")
print("-" * 60)


# ------------------------------------------------------
# STEP 4: Train the model
# ------------------------------------------------------
# epochs: number of times the model sees the full dataset
# verbose=1 shows training progress; use verbose=0 to hide it
print("STEP 4: Training...")
history = model.fit(X, y, epochs=200, verbose=0)
print("Training complete.")
print("-" * 60)


# ------------------------------------------------------
# STEP 5: Evaluate on the same tiny dataset (just for demo)
# ------------------------------------------------------
loss, acc = model.evaluate(X, y, verbose=0)
print("STEP 5: Evaluation on training data")
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {acc:.4f}")
print("-" * 60)


# ------------------------------------------------------
# STEP 6: Make predictions on NEW students
# ------------------------------------------------------
# IMPORTANT: Input to model.predict must be 2D: (batch, features)
# So for ONE student, shape should be (1, 4)
new_students = tf.constant([
    [75, 80, 70, 85],   # likely Pass
    [55, 50, 52, 65],   # likely Fail/Borderline
], dtype=tf.float32)

print("STEP 6: Prediction input shape:", new_students.shape)

pred_probs = model.predict(new_students, verbose=0)  # probabilities between 0 and 1

# Convert probability to Pass/Fail using threshold 0.5
pred_labels = (pred_probs >= 0.5).astype(int)

for i, (prob, label) in enumerate(zip(pred_probs, pred_labels), start=1):
    print(f"Student {i}: pass_probability={prob[0]:.3f} -> predicted_label={label[0]}")

print("-" * 60)


# ------------------------------------------------------
# STEP 7 (Optional): Show learned weights (Dense layers store weights)
# ------------------------------------------------------
# This is optional but useful to show trainees that the model "learned numbers".
print("STEP 7 (Optional): Learned weights in each Dense layer")
for layer in model.layers:
    if isinstance(layer, Dense):
        W, b = layer.get_weights()
        print(f"\nLayer: {layer.name}")
        print("Weights shape:", W.shape)
        print("Bias shape:", b.shape)


STEP 1: Data shapes
X shape: (226, 4)
y shape: (226, 1)
------------------------------------------------------------
STEP 2: Model summary (shows shape flow)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 8)              │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 121 (484.00 B)

 Trainable params: 121 (484.00 B)

 Non-trainable params: 0 (0.00 B)

------------------------------------------------------------
STEP 3: Model compiled
------------------------------------------------------------
STEP 4: Training...
Training complete.
------------------------------------------------------------
STEP 5: Evaluation on training data
Loss: 0.3908
Accuracy: 0.8363
------------------------------------------------------------
STEP 6: Prediction input shape: (2, 4)
Student 1: pass_probability=0.598 -> predicted_label=1
Student 2: pass_probability=0.225 -> predicted_label=0
------------------------------------------------------------
STEP 7 (Optional): Learned weights in each Dense layer

Layer: dense
Weights shape: (4, 8)
Bias shape: (8,)

Layer: dense_1
Weights shape: (8, 8)
Bias shape: (8,)

Layer: dense_2
Weights shape: (8, 1)
Bias shape: (1,)


In [13]:
test_student = tf.constant(
    [[85, 88, 82, 92]],  # High scores & attendance → should PASS
    dtype=tf.float32
)

print("STEP 8: Testing with a single student")
print("Test student input shape:", test_student.shape)

# Get prediction (probability)
test_prediction = model.predict(test_student, verbose=0)

# Interpret prediction
pass_probability = test_prediction[0][0]
predicted_label = 1 if pass_probability >= 0.5 else 0

print(f"Predicted pass probability: {pass_probability:.3f}")
print("Predicted result:", "PASS ✅" if predicted_label == 1 else "FAIL ❌")
print("-" * 60)

STEP 8: Testing with a single student
Test student input shape: (1, 4)
Predicted pass probability: 0.678
Predicted result: PASS ✅
------------------------------------------------------------


In [56]:
# Adding layers cannot garuntee accuracy. Data quality is most important for optimal results